In [1]:
import os
os.chdir('/mnt/c/Users/ralvin/OneDrive - Reliant Health Partners/Documents/RHP_dev_RA/Automate_skyvia')
import pandas as pd
from simple_salesforce import Salesforce, SalesforceLogin
import pyodbc, sys, time 
import requests
import numpy as np
from datetime import datetime
from helper_functions_v3 import create_df, updated_data_pull, upsert, delete_record, insert_records, salesforce_connection, read_sftp_data, data_pull
import configparser
import json
import paramiko
import io

pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 


##### retrieve source data
config = configparser.ConfigParser()
config.read('config.ini')
sftp_config = config['sftp']
mapping_config = config['mappings']
host = sftp_config.get('host')
port = sftp_config.getint('port', fallback=22) 
username = sftp_config.get('user')
password = sftp_config.get('password')
remote_path = sftp_config.get('remote_path')
json_path = mapping_config.get('json_file_path')


Source_data =  read_sftp_data(remote_path, host, port, username, password)
print(f'Source data shape: {Source_data.shape}')


## get data mappings 
with open(json_path, 'r') as f:
    data = json.load(f)

mappings = {k:v for k,v in data.items()}
Claims_map = mappings['Claim_Object_Map']
Account_map = mappings['Account_Object_Map']
Povider_map = mappings['Provider_Object_Map']
lines_map = mappings['Line_Item_Object_Map']

## sf connection
sf = salesforce_connection(sandbox=True)
sf_prod = salesforce_connection(sandbox=False)





###upsert account and provider records
map_account_table, _ = create_df(Account_map, Source_data, sf)
map_provider_table, keys = create_df(Povider_map, Source_data, sf)
new_ids, errors = upsert(map_provider_table.drop_duplicates(), keys, sf)

##### populate claims obj on sf
map_claims_table, keys = create_df(Claims_map, Source_data, sf)
new_ids_claims, errors = insert_records(map_claims_table.drop_duplicates(), keys, sf) ###insert new claims records

##### populate lines obj on sf
map_lines_table, keys = create_df(lines_map, Source_data, sf, new_ids_claims)
new_ids_lines, errors = insert_records(map_lines_table.drop_duplicates(), keys, sf) ###insert new line records


Connection established successfully!
File read into DataFrame successfully
Connection closed.
Source data shape: (446, 42)
Connected to Salesforce sandbox
Connected to Salesforce Prod
pulling updated Provider_TIN__c table
pulled Provider_TIN__c
Merged TIN
pulling updated Groups_Clients__c table
pulled Groups_Clients__c
Merged Patient Group/Policy Number
pulling updated Jurisdiction__c table
pulled Jurisdiction__c
Merged JurisdictionState
pulling updated DRG__c table
pulled DRG__c
Merged DRG
pulling updated Provider_Specialty__c table
pulled Provider_Specialty__c
Merged Billing Provider Taxonomy
Processing record 0: {'Claim_ID__c': 'A3038B5E4B6C914tksft', 'Provider_TIN__c': 'a0FVF000001zh5t2AA', 'Group_Client__c': 'a0C8a00000r3GJ6EAM', 'Claim_Number__c': '431817310700011', 'Jurisdiction__c': 'a024W00000ICbumQAD', 'Diag_Code__c': 'M25562', 'Diag_Code_2nd__c': nan, 'Diag_Code_3rd__c': nan, 'Diag_Code_4th__c': nan, 'Provider__c': 'KAISER FOUNDATION HOSPITALS ', 'Patient__c': 'WATKINS MONIC

In [9]:
sf = salesforce_connection(sandbox=True)


claims_sf = updated_data_pull(['Claims__c'], sf)
Line_items_sf = updated_data_pull(['Line_Items__c'], sf)
print(claims_sf.shape)
print(Line_items_sf.shape)

Connected to Salesforce sandbox
pulled Claims__c
pulled Line_Items__c
(12937, 101)
(22841, 49)


In [8]:
### delete records

source_cases = Source_data.drop_duplicates(subset=['Claim ID'])[['Claim ID']]
ids = list(source_cases.merge(updated_data_pull(['Claims__c'], sf), left_on='Claim ID', right_on='Claim_ID__c').loc[:, ['Claim ID', 'Claim_ID__c', 'Id']]['Id'])
keys = {'Claims__c':'Claim_ID__c'}
delete_record(ids, keys, sf)

pulled Claims__c
Object: Claims__c, Records to process: 103
All records deleted successfully.


In [1]:
import os
os.chdir('/mnt/c/Users/ralvin/OneDrive - Reliant Health Partners/Documents/RHP_dev_RA/Automate_skyvia')
import pandas as pd
from simple_salesforce import Salesforce, SalesforceLogin
import pyodbc, sys, time 
import requests
import numpy as np
from datetime import datetime
from helper_functions_v3 import create_df, updated_data_pull, upsert, delete_record, insert_records, salesforce_connection, read_sftp_data, read_sftp_directory, data_pull
import configparser
import json
import paramiko
import io

pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 


##### retrieve source data
config = configparser.ConfigParser()
config.read('config_v2.ini')
sftp_config = config['sftp']
mapping_config = config['mappings']
host = sftp_config.get('host')
port = sftp_config.getint('port', fallback=22) 
username = sftp_config.get('user')
password = sftp_config.get('password')
remote_dir = sftp_config.get('remote_path')
json_path = mapping_config.get('json_file_path')



Source_data = read_sftp_directory(remote_dir, host, port, username, password) ### list of all source dataframes
for i in Source_data:
    print(i)
    print(f'Source data shape: {Source_data[i].shape}')

Connection established successfully!
Files in directory: ['837I00111182024.csv', '837P00111182024.csv']
File 837I00111182024.csv read into DataFrame successfully
File 837P00111182024.csv read into DataFrame successfully
Connection closed.
837I00111182024.csv
Source data shape: (446, 42)
837P00111182024.csv
Source data shape: (3106, 42)


In [3]:
Source_data['837P00111182024.csv'].drop_duplicates().shape

(3106, 42)

In [5]:

sf = salesforce_connection(sandbox=True)
with open(json_path, 'r') as f:
    data = json.load(f)

mappings = {k:v for k,v in data.items()}
Claims_map = mappings['Claim_Object_Map']
Account_map = mappings['Account_Object_Map']
Povider_map = mappings['Provider_Object_Map']
lines_map = mappings['Line_Item_Object_Map']


map_claims_table, keys = create_df(Claims_map, Source_data['837P00111182024.csv'], sf)

Connected to Salesforce sandbox
pulling updated Provider_TIN__c table
pulled Provider_TIN__c
Merged TIN
pulling updated Groups_Clients__c table
pulled Groups_Clients__c
Merged Patient Group/Policy Number
pulling updated Jurisdiction__c table
pulled Jurisdiction__c
Merged JurisdictionState
pulling updated DRG__c table
pulled DRG__c
Merged DRG
pulling updated Provider_Specialty__c table
pulled Provider_Specialty__c
Merged Billing Provider Taxonomy


In [9]:
map_claims_table.drop_duplicates(subset=['Claim_ID__c']).shape

(1304, 20)

In [8]:
map_claims_table.head()

,Claim_ID__c,Provider_TIN__c,Group_Client__c,Claim_Number__c,Jurisdiction__c,Diag_Code__c,Diag_Code_2nd__c,Diag_Code_3rd__c,Diag_Code_4th__c,Provider__c,Patient__c,Patient_ID__c,Date_of_Birth__c,Provider_Zip__c,DRG__c,QPA__c,NPI__c,POS__c,Provider_Specialty__c,Service_Type__c
0,BFC71C030596B19tksft,a0F4W00000W1Kb7UAF,a0C8a00000r3Gf3EAE,243190499E,a024W00000HYAjjQAH,I10,Z125,E785,E119,PATHGROUP LABS LLC,MCHANEY ERIC C,MB04526570,10/09/1972,372175200,NaN,N,1.235187e+09,81,a008a000015yXORAA2,LAB
1,BFC71C030596B19tksft,a0F4W00000W1Kb7UAF,a0C8a00000r3Gf3EAE,243190499E,a024W00000HYAjjQAH,I10,Z125,E785,E119,PATHGROUP LABS LLC,MCHANEY ERIC C,MB04526570,10/09/1972,372175200,NaN,N,1.235187e+09,81,a008a000015yXORAA2,LAB
2,BFC71C030596B19tksft,a0F4W00000W1Kb7UAF,a0C8a00000r3Gf3EAE,243190499E,a024W00000HYAjjQAH,I10,Z125,E785,E119,PATHGROUP LABS LLC,MCHANEY ERIC C,MB04526570,10/09/1972,372175200,NaN,N,1.235187e+09,81,a008a000015yXORAA2,LAB
3,BFC71C030596B19tksft,a0F4W00000W1Kb7UAF,a0C8a00000r3Gf3EAE,243190499E,a024W00000HYAjjQAH,I10,Z125,E785,E119,PATHGROUP LABS LLC,MCHANEY ERIC C,MB04526570,10/09/1972,372175200,NaN,N,1.235187e+09,81,a008a000015yXORAA2,LAB
4,BFC71C030596B19tksft,a0F4W00000W1Kb7UAF,a0C8a00000r3Gf3EAE,243190499E,a024W00000HYAjjQAH,I10,Z125,E785,E119,PATHGROUP LABS LLC,MCHANEY ERIC C,MB04526570,10/09/1972,372175200,NaN,N,1.235187e+09,81,a008a000015yXORAA2,LAB
